In [1]:
import pandas as pd
import numpy as np

In [2]:
referrals = pd.read_csv('../../Welcome-Centre-DataCorps-Data/referrals.csv')

/Users/jamesgin/.virtualenvs/datasine/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (44,46,47,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
referrals['ReferralTakenDate'] = pd.to_datetime(referrals['ReferralTakenDate'])

In [16]:
test = referrals[referrals['ClientId'] == 594].sort_values('ReferralTakenDate')

In [17]:
rest_days = 28
days = (test['ReferralTakenDate'] - test['ReferralTakenDate'].min()).dt.days
days.diff()

936        NaN
937        8.0
938       16.0
939       32.0
1598      39.0
2967      45.0
1662      10.0
4702     116.0
4904      14.0
4908       1.0
5370      17.0
5621      17.0
6032      30.0
6331      20.0
6561      14.0
6891      22.0
7684      62.0
7924      20.0
8789      72.0
8817       1.0
8971      11.0
9268      23.0
9331       6.0
9361       1.0
9803      33.0
9898       4.0
10615     45.0
11334     51.0
11696     26.0
12388     36.0
12809     34.0
13784     77.0
14604     58.0
14790     13.0
15213     27.0
15325      7.0
15347      1.0
16787     90.0
17199     17.0
17868     40.0
Name: ReferralTakenDate, dtype: float64

In [41]:
def get_burst_number(group, rest_days):
    days = (group['ReferralTakenDate'] - group['ReferralTakenDate'].min()).dt.days
    same_burst = (days.diff() < rest_days)
    burst_number = (((same_burst != same_burst.shift(1)) & ~same_burst) | ~same_burst).cumsum()
    return burst_number

In [42]:
def add_burst_numbers(referrals, rest_days=28):
    referrals = referrals.sort_values('ReferralTakenDate')
    burst_number = referrals.groupby(referrals['ClientId']).apply(get_burst_number, rest_days)
#     return burst_number
    referrals['burst_number'] = burst_number.reset_index().set_index('level_1')['ReferralTakenDate']
    return referrals

In [43]:
burst_number = add_burst_numbers(referrals,28)

In [45]:
burst_number[burst_number['ClientId'] == 594]['burst_number']

936       1
937       1
938       1
939       2
1598      3
2967      4
1662      4
4702      5
4904      5
4908      5
5370      5
5621      5
6032      6
6331      6
6561      6
6891      6
7684      7
7924      7
8789      8
8817      8
8971      8
9268      8
9331      8
9361      8
9803      9
9898      9
10615    10
11334    11
11696    11
12388    12
12809    13
13784    14
14604    15
14790    15
15213    15
15325    15
15347    15
16787    16
17199    16
17868    17
Name: burst_number, dtype: int64

In [26]:
group = test
days = (group['ReferralTakenDate'] - group['ReferralTakenDate'].min()).dt.days
same_burst = (days.diff().fillna(0) < rest_days)
burst_number = (same_burst != same_burst.shift(-1).fillna(True)).cumsum()

In [40]:
pd.concat([days.diff().fillna(0), same_burst, same_burst.shift(1), same_burst.shift(-1), 
          (((same_burst != same_burst.shift(1)) & ~same_burst) | ~same_burst).cumsum()], axis=1)

,ReferralTakenDate,ReferralTakenDate,ReferralTakenDate,ReferralTakenDate,ReferralTakenDate
936,0.0,True,NaN,True,0
937,8.0,True,True,True,0
938,16.0,True,True,False,0
939,32.0,False,True,False,1
1598,39.0,False,False,False,2
2967,45.0,False,False,True,3
1662,10.0,True,False,False,3
4702,116.0,False,True,True,4
4904,14.0,True,False,True,4
4908,1.0,True,True,True,4


In [38]:
((same_burst != same_burst.shift(1)) | ~same_burst).cumsum()

936       1
937       1
938       1
939       2
1598      3
2967      4
1662      5
4702      6
4904      7
4908      7
5370      7
5621      7
6032      8
6331      9
6561      9
6891      9
7684     10
7924     11
8789     12
8817     13
8971     13
9268     13
9331     13
9361     13
9803     14
9898     15
10615    16
11334    17
11696    18
12388    19
12809    20
13784    21
14604    22
14790    23
15213    23
15325    23
15347    23
16787    24
17199    25
17868    26
Name: ReferralTakenDate, dtype: int64

In [24]:
same_burst != same_burst.shift(-1)

936       True
937      False
938       True
939      False
1598     False
2967      True
1662      True
4702      True
4904     False
4908     False
5370     False
5621      True
6032      True
6331     False
6561     False
6891      True
7684      True
7924      True
8789      True
8817     False
8971     False
9268     False
9331     False
9361      True
9803      True
9898      True
10615    False
11334     True
11696     True
12388    False
12809    False
13784    False
14604     True
14790    False
15213    False
15325    False
15347     True
16787     True
17199     True
17868     True
Name: ReferralTakenDate, dtype: bool

In [13]:
burst_number[referrals['ClientId'] == 594]['burst_number']

/Users/jamesgin/.virtualenvs/datasine/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


936       1
937       1
938       2
939       2
1598      2
2967      3
1662      4
4702      5
4904      5
4908      5
5370      5
5621      6
6032      7
6331      7
6561      7
6891      8
7684      9
7924     10
8789     11
8817     11
8971     11
9268     11
9331     11
9361     12
9803     13
9898     14
10615    14
11334    15
11696    16
12388    16
12809    16
13784    16
14604    17
14790    17
15213    17
15325    17
15347    18
16787    19
17199    20
17868    21
Name: burst_number, dtype: int64

In [97]:
burst_number.to_csv('with_burst_number.csv')

In [82]:
burst_number.groupby(['ClientId', 'burst_number']).size().reset_index().groupby(['ClientId']).size().idxmax()

594

In [99]:
test

,ReferralInstanceId,StatusId,ReferralOnHold,ReferralTakenDate,ReferralReadyDate,ReferralCollectedDate,ReferralWorkerID,ReferralPreparedWorkerId,ReferralHandedWorkerId,ClientId,...,ReferralDietaryRequirements_Diabetic,ReferralDietaryRequirements_Gluten Intolerant/coeliac,ReferralDietaryRequirements_Halal,ReferralDietaryRequirements_No Pork,ReferralDietaryRequirements_Nut Allergy,ReferralDietaryRequirements_Other Dietary Requirement,ReferralDietaryRequirements_Vegan,ReferralDietaryRequirements_Vegetarian,ReferralDietaryRequirements_lactose intolerent,ReferralDietaryRequirements_pregnancy
936,937,3,0,2014-02-03 00:00:00,02/03/14 00:00:00,02/03/14 00:00:00,4,NaN,NaN,594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937,938,3,0,2014-02-11 00:00:00,02/11/14 00:00:00,02/11/14 00:00:00,4,NaN,NaN,594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,939,3,0,2014-02-27 00:00:00,02/27/14 00:00:00,02/27/14 00:00:00,4,NaN,NaN,594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
939,940,3,0,2014-03-31 00:00:00,03/31/14 00:00:00,03/31/14 00:00:00,4,NaN,NaN,594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1598,2186,3,0,2014-05-09 15:30:00,05/09/14 15:30:00,05/09/14 15:30:00,7,7.0,7.0,594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1662,3008,3,0,2014-07-03 15:09:00,07/03/14 15:09:00,07/03/14 15:09:00,7,7.0,7.0,594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2967,2859,3,0,2014-06-23 15:20:00,06/23/14 15:20:00,06/23/14 15:20:00,7,7.0,7.0,594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4702,4703,3,0,2014-10-27 14:08:00,10/27/14 15:31:00,10/27/14 15:31:00,5,5.0,5.0,594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4904,4905,3,0,2014-11-10 16:24:00,11/10/14 16:24:00,11/10/14 16:24:00,1,1.0,1.0,594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4908,4909,3,0,2014-11-11 10:43:00,11/11/14 14:28:00,11/11/14 14:28:00,1,5.0,5.0,594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
test = referrals[referrals['ClientId'] == 594]
%matplotlib inline
test['ReferrralTakenDate']

ValueError: scatter requires x column to be numeric

In [75]:
burst_number.reset_index().set_index('level_1')['ReferralTakenDate']

level_1
0        0
1        1
2        1
1860     1
1986     1
2269     1
2396     1
3        1
4        1
5        1
6        0
5653     1
8        1
9        0
6897     0
7395     1
10       1
11       1
12       2
2739     3
2841     3
2981     4
15005    5
15342    5
15763    5
13       1
14       1
15       1
17       0
9189     1
        ..
17847    1
17851    1
17979    1
17853    1
17862    1
17860    1
17864    1
17871    1
17874    1
17947    1
17881    1
17882    1
17883    1
17892    1
17893    1
17896    1
17903    1
17905    1
17907    1
17912    1
17916    1
17918    1
17924    1
17932    1
17937    1
17948    1
17952    1
17959    1
17968    1
17969    1
Name: ReferralTakenDate, Length: 17980, dtype: int64

In [49]:
referrals.pipe(add_burst_numbers, 28)

TypeError: incompatible index of inserted column with frame index

In [38]:
burst_number

0       0
1       1
2       1
1860    1
1986    1
2269    1
2396    1
Name: ReferralTakenDate, dtype: int64

In [34]:
(same_burst != same_burst.shift(-1).fillna(True))

0       False
1        True
2       False
1860    False
1986    False
2269    False
2396    False
Name: ReferralTakenDate, dtype: bool

In [26]:
same_burst

0       False
1       False
2        True
1860     True
1986     True
2269     True
2396     True
Name: ReferralTakenDate, dtype: bool